<a href="https://colab.research.google.com/github/aneesh-shetye/NLP/blob/master/char_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch 
import torch.nn as nn
import numpy as np
from google.colab import drive
import random

device = torch.device(type='cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
data = open('dino_names.txt', 'r').read() 
data = data.lower()
data = data.replace("\n", ".")
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)

print("no of data = " ,data_size)
print("no of unique chars = " ,vocab_size)

char_to_ix = {ch:i for i,ch in enumerate(chars)}   
ix_to_char = {i:ch for i,ch in enumerate(chars)}    

data = data.replace(".", " ")
data = data.split()
random.shuffle(data)

no of data =  19914
no of unique chars =  28


In [ ]:
class rnn(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(rnn, self).__init__()

        self.hidden_size = hidden_size
        self.input_size = input_size 
        self.output_size = input_size # its a predective char rnn

        self.e2i = nn.Linear(input_size, input_size)
        self.i2h = nn.Linear(hidden_size + input_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, self.output_size)
        self.o2o = nn.Linear(self.output_size, self.output_size)
        self.tanh = nn.Tanh()
        #self.softmax = nn.Softmax(dim=1)

    def forward(self, input, hidden):

        input = self.e2i(input)
        input_hidd = torch.cat((input,hidden), dim=1)
        hidden = self.i2h(input_hidd)
        hidden = self.tanh(hidden)
        output = self.h2o(hidden)
        output = self.o2o(output)
        #output = self.softmax(output)

        return hidden , output 

In [ ]:
learning_rate = 0.0005

model = rnn(input_size=vocab_size, hidden_size=75)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train(data, hidden_size, vocab_size):

    epoch_loss = 0
    no = 0
    for word in data:

        total_loss = 0

        chars_in_data = list(word)
        length_of_word = len(chars_in_data)

        hidden = torch.zeros([1, hidden_size])
        hidden = hidden.to(device)

        predictions = torch.zeros([1,vocab_size])
        predictions[0,char_to_ix[chars_in_data[0]]] = 1
        predictions = predictions.to(device)

        target = [char_to_ix[i]  for i in chars_in_data]
        target.append(char_to_ix["."]) 
        target_tensor = torch.LongTensor(target)

        for char in chars_in_data:
            
            input_idx = char_to_ix[char]
            input = torch.zeros([1, vocab_size], dtype=torch.float)
            input = input.to(device)
            input[0, input_idx] = 1

            hidden, output = model(input, hidden)

            predictions = torch.cat((predictions, output), dim=0)
            predictions = predictions.to(device)
            


        batch_loss = criterion(predictions, target_tensor)
        no+=1
        total_loss += batch_loss
        epoch_loss+= batch_loss 

        optimizer.zero_grad()
        batch_loss.backward(retain_graph=True)
        optimizer.step()
    
    epoch_loss = epoch_loss/no

    print(epoch_loss)


In [ ]:
for _ in range(15):
    train(data=data, hidden_size=75, vocab_size=vocab_size)

tensor(2.2810, grad_fn=<DivBackward0>)
tensor(1.9582, grad_fn=<DivBackward0>)
tensor(1.8863, grad_fn=<DivBackward0>)
tensor(1.8387, grad_fn=<DivBackward0>)
tensor(1.7996, grad_fn=<DivBackward0>)
tensor(1.7642, grad_fn=<DivBackward0>)
tensor(1.7346, grad_fn=<DivBackward0>)
tensor(1.7097, grad_fn=<DivBackward0>)
tensor(1.6877, grad_fn=<DivBackward0>)
tensor(1.6677, grad_fn=<DivBackward0>)
tensor(1.6491, grad_fn=<DivBackward0>)
tensor(1.6315, grad_fn=<DivBackward0>)
tensor(1.6148, grad_fn=<DivBackward0>)
tensor(1.5992, grad_fn=<DivBackward0>)
tensor(1.5845, grad_fn=<DivBackward0>)


In [ ]:
char_to_ix["."]

18

In [ ]:

model.eval()
hidden_size = 75

for letters in chars:

    print(letters,":", end = " ")
    print(letters, end="")
    char = letters
    input_idx = char_to_ix[char]
    input = torch.zeros([1, vocab_size], dtype=torch.float)
    input = input.to(device)
    input[0, input_idx] = 1

    hidden = torch.zeros([1, hidden_size])
    hidden = hidden.to(device)
    idx = torch.tensor(char_to_ix[letters])

    while ix_to_char[idx.item()] != ".":
        
        hidden, output = model(input, hidden)
        
        output = nn.Softmax(dim=1)(output)
        idx = output.argmax()
        input = torch.zeros([1, vocab_size], dtype=torch.float)
        input[0, idx] = 1
        print(ix_to_char[idx.item()], end="")

    print("\n")

m : manganosaurus.

x : xiangosaurus.

n : notosaurus.

k : kararosaurus.

l : langanosaurus.

s : saurosaurus.

u : urophosaurus.

a : altarosaurus.

p : palarosaurus.

o : ornithosaurus.

y : yunganosaurus.

i : inosaurus.

h : harahosaurus.

v : varasaurus.

z : zenganosaurus.

t : taratosaurus.

w : walahosaurus.

j : jiangosaurus.

. : .

g : gonganosaurus.

c : chengosaurus.

f : furopelta.

b : baratosaurus.

  :  angangosaurus.

r : rarosaurus.

d : danganosaurus.

q : quangosaurus.

e : eratosaurus.



Using Preprovided RNN, LSTM and GRU